In [5]:
import pandas as pd
from IPython.display import display

from xcai.basics import *
from xcai.plot import *

import matplotlib.pyplot as plt

### Helper function

In [2]:
set_plot_defaults()

In [3]:
def metrics(a,b,c,x,y):
    return {'P@1':a, 'P@5':b,'N@5':c, 'PSP@1':x, 'PSP@5':y}

def display_results(metric, index):
    with pd.option_context('display.precision', 2):
        display(pd.DataFrame(metric, index=index))

### `Teacher`

In [7]:
# wikiseealsotitles
m = {'test_loss': 0.016681302338838577, 'test_P@1': 0.47479367940737405, 'test_P@10': 0.14173393797699302, 'test_P@3': 0.30295280211078307, 'test_P@5': 0.22635495591925775, 'test_N@1': 0.4747936725616455, 'test_N@10': 0.5081887245178223, 'test_N@3': 0.46679285168647766, 'test_N@5': 0.48181095719337463, 'test_PSP@1': 0.3659764403986233, 'test_PSP@10': 0.47102664459277543, 'test_PSP@3': 0.38466570585689525, 'test_PSP@5': 0.4126957923700924, 'test_PSN@1': 0.3659764528274536, 'test_PSN@10': 0.4537607729434967, 'test_PSN@3': 0.3976207375526428, 'test_PSN@5': 0.42237570881843567, 'test_R@200': 0.7736651615675022, 'test_R@10': 0.5737313838358737, 'test_R@100': 0.7436145963127184, 'test_runtime': 239.3287, 'test_samples_per_second': 741.721, 'test_steps_per_second': 0.464}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,47.4794,30.2953,22.6355,14.1734,47.4794,46.6793,48.1811,50.8189,36.5976,38.4666,41.2696,47.1027,36.5976,39.7621,42.2376,45.3761,57.3731,74.3615,77.3665,0.0167,239.3287,741.721,0.464


In [10]:
# wikititles
m = {'test_loss': 0.017709823325276375, 'test_P@1': 0.6604039844693987, 'test_P@10': 0.1922625146260841, 'test_P@3': 0.4427169790774948, 'test_P@5': 0.3219657464253877, 'test_N@1': 0.6604039669036865, 'test_N@10': 0.5267409682273865, 'test_N@3': 0.5628888010978699, 'test_N@5': 0.534044623374939, 'test_PSP@1': 0.38866437063552794, 'test_PSP@10': 0.45216630925636925, 'test_PSP@3': 0.41840363402702435, 'test_PSP@5': 0.4250382267401933, 'test_PSN@1': 0.3886643648147583, 'test_PSN@10': 0.47987130284309387, 'test_PSN@3': 0.42873772978782654, 'test_PSN@5': 0.4506883919239044, 'test_R@200': 0.702557668379058, 'test_R@10': 0.510692117939397, 'test_R@100': 0.663280306014536, 'test_runtime': 580.8063, 'test_samples_per_second': 1349.405, 'test_steps_per_second': 0.844}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,66.0404,44.2717,32.1966,19.2263,66.0404,56.2889,53.4045,52.6741,38.8664,41.8404,42.5038,45.2166,38.8664,42.8738,45.0688,47.9871,51.0692,66.328,70.2558,0.0177,580.8063,1349.405,0.844


In [7]:
# wikiseealso
m = {'test_loss': 0.015018783509731293, 'test_P@1': 0.5037095456721967, 'test_P@10': 0.15165478973596774, 'test_P@3': 0.3256964200210398, 'test_P@5': 0.2433811227220156, 'test_N@1': 0.5037095546722412, 'test_N@10': 0.541210949420929, 'test_N@3': 0.4979328513145447, 'test_N@5': 0.5138446688652039, 'test_PSP@1': 0.3672674882608596, 'test_PSP@10': 0.49842471202063104, 'test_PSP@3': 0.4018981757157289, 'test_PSP@5': 0.43512711505254204, 'test_PSN@1': 0.3672674894332886, 'test_PSN@10': 0.474685400724411, 'test_PSN@3': 0.41203731298446655, 'test_PSN@5': 0.4404144287109375, 'test_R@200': 0.8056290164783148, 'test_R@10': 0.6113826946844456, 'test_R@100': 0.7781915615151795, 'test_runtime': 316.9593, 'test_samples_per_second': 560.056, 'test_steps_per_second': 0.35}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,50.371,32.5696,24.3381,15.1655,50.371,49.7933,51.3845,54.1211,36.7267,40.1898,43.5127,49.8425,36.7267,41.2037,44.0414,47.4685,61.1383,77.8192,80.5629,0.015,316.9593,560.056,0.35


In [8]:
# wikipedia
m = {'test_loss': 0.010197877883911133, 'test_P@1': 0.8240430855522792, 'test_P@10': 0.3066724678888317, 'test_P@3': 0.6387183043424678, 'test_P@5': 0.4950069091535431, 'test_N@1': 0.8240430951118469, 'test_N@10': 0.7470307946205139, 'test_N@3': 0.7715214490890503, 'test_N@5': 0.750892162322998, 'test_PSP@1': 0.4637808932625801, 'test_PSP@10': 0.6563743396782328, 'test_PSP@3': 0.5579214660003528, 'test_PSP@5': 0.5994651830178395, 'test_PSN@1': 0.46378087997436523, 'test_PSN@10': 0.6357831954956055, 'test_PSN@3': 0.5512413382530212, 'test_PSN@5': 0.5932626128196716, 'test_R@200': 0.8997065010083698, 'test_R@10': 0.7399693088722892, 'test_R@100': 0.8772790846554102, 'test_runtime': 658.7976, 'test_samples_per_second': 1189.657, 'test_steps_per_second': 0.372}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,82.4043,63.8718,49.5007,30.6672,82.4043,77.1521,75.0892,74.7031,46.3781,55.7921,59.9465,65.6374,46.3781,55.1241,59.3263,63.5783,73.9969,87.7279,89.9707,0.0102,658.7976,1189.657,0.372


### `momos M1`

In [9]:
# wikiseealsotitles
m = {'test_loss': 0.20433445274829865, 'test_P@1': 0.3429512998901501, 'test_P@10': 0.11108300706984563, 'test_P@3': 0.23351829422867218, 'test_P@5': 0.17679632707113338, 'test_N@1': 0.34295129776000977, 'test_N@10': 0.37233859300613403, 'test_N@3': 0.3428209722042084, 'test_N@5': 0.35341933369636536, 'test_PSP@1': 0.27993850226241385, 'test_PSP@10': 0.376466313139454, 'test_PSP@3': 0.30754761672224956, 'test_PSP@5': 0.3316568403739362, 'test_PSN@1': 0.2799384891986847, 'test_PSN@10': 0.34672781825065613, 'test_PSN@3': 0.30699291825294495, 'test_PSN@5': 0.3245813846588135, 'test_R@200': 0.5835736040171667, 'test_R@10': 0.4208163969078128, 'test_R@100': 0.5554816765045649, 'test_runtime': 280.87, 'test_samples_per_second': 632.018, 'test_steps_per_second': 0.395}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,34.2951,23.3518,17.6796,11.1083,34.2951,34.2821,35.3419,37.2339,27.9939,30.7548,33.1657,37.6466,27.9938,30.6993,32.4581,34.6728,42.0816,55.5482,58.3574,0.2043,280.87,632.018,0.395


In [6]:
#wikiseealsotitles

# test_enc + 0.2 * test_teacher_lbl
{'P@1': 0.34457933132411345, 'P@10': 0.11202489930430193, 'P@3': 0.23517261452085578, 'P@5': 0.17794777906110715, 'N@1': 0.34457934, 'N@10': 0.37385765, 'N@3': 0.3442961, 'N@5': 0.35469761, 'PSP@1': 0.27901799771375035, 'PSP@10': 0.3792134699643732, 'PSP@3': 0.30876249855090515, 'PSP@5': 0.33322251235980027, 'PSN@1': 0.27901798, 'PSN@10': 0.34732234, 'PSN@3': 0.30716676, 'PSN@5': 0.32484227, 'R@200': 0.5852598297168966, 'R@10': 0.42251354813475966, 'R@100': 0.561321716537376}
display_metric(m, remove_prefix=False)

# test_enc + 0.3 * test_centroid_repr
m = {'P@1': 0.3472889614962116, 'P@10': 0.11220178576460135, 'P@3': 0.23585424705895192, 'P@5': 0.178364645241428, 'N@1': 0.34728897, 'N@10': 0.37524232, 'N@3': 0.3459366, 'N@5': 0.3562202, 'PSP@1': 0.2836158698437905, 'PSP@10': 0.38013037113037834, 'PSP@3': 0.3111235493277344, 'PSP@5': 0.3349052664347519, 'PSN@1': 0.28361586, 'PSN@10': 0.3493827, 'PSN@3': 0.310036, 'PSN@5': 0.32726714, 'R@200': 0.5847146349994797, 'R@10': 0.4227361736324652, 'R@100': 0.5574637129920544}
display_metric(m, remove_prefix=False)

# test_clf + 0.5 * test_centroid_fused_repr
m = {'P@1': 0.3486972931864913, 'P@10': 0.1130129848181999, 'P@3': 0.23705602343465887, 'P@5': 0.17949018392829996, 'N@1': 0.3486973, 'N@10': 0.3771655, 'N@3': 0.34739548, 'N@5': 0.35785893, 'PSP@1': 0.28390518763666195, 'PSP@10': 0.38173471548426585, 'PSP@3': 0.31175215012557717, 'PSP@5': 0.3360194378754333, 'PSN@1': 0.28390518, 'PSN@10': 0.350171, 'PSN@3': 0.3104189, 'PSN@5': 0.32781738, 'R@200': 0.588366797901596, 'R@10': 0.425270609770189, 'R@100': 0.5596008034348555}
display_metric(m, remove_prefix=False)

,test_P@1,test_P@3,test_P@5,test_P@10,test_N@1,test_N@3,test_N@5,test_N@10,test_PSP@1,test_PSP@3,test_PSP@5,test_PSP@10,test_PSN@1,test_PSN@3,test_PSN@5,test_PSN@10,test_R@10,test_R@100,test_R@200,test_loss,test_runtime,test_samples_per_second,test_steps_per_second
0,48.3897,31.8118,23.7779,14.7284,48.3897,48.0932,49.5622,52.0498,34.7167,38.7188,42.0607,47.9726,34.7167,39.2589,41.9797,45.141,58.5767,73.8925,76.5243,0.1638,354.4095,500.875,0.313


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,34.7289,23.5854,17.8365,11.2202,34.7289,34.5937,35.622,37.5242,28.3616,31.1124,33.4905,38.013,28.3616,31.0036,32.7267,34.9383,42.2736,55.7464,58.4715


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,34.8697,23.7056,17.949,11.3013,34.8697,34.7395,35.7859,37.7165,28.3905,31.1752,33.6019,38.1735,28.3905,31.0419,32.7817,35.0171,42.5271,55.9601,58.8367


In [3]:
# wikititles
m = {'test_loss': 0.06982655078172684, 'test_P@1': 0.46251003198752655, 'test_P@10': 0.09739276778217673, 'test_P@3': 0.24712224968344648, 'test_P@5': 0.16913911830710657, 'test_N@1': 0.46251001954078674, 'test_N@10': 0.3304288387298584, 'test_N@3': 0.34725072979927063, 'test_N@5': 0.3298164904117584, 'test_PSP@1': 0.2686345235323221, 'test_PSP@10': 0.248760120692636, 'test_PSP@3': 0.2503089653047077, 'test_PSP@5': 0.24214672863904063, 'test_PSN@1': 0.26863452792167664, 'test_PSN@10': 0.31364181637763977, 'test_PSN@3': 0.27593907713890076, 'test_PSN@5': 0.2913457155227661, 'test_R@200': 0.4521662086595291, 'test_R@10': 0.3218104563670417, 'test_R@100': 0.421645572522769, 'test_runtime': 1559.4997, 'test_samples_per_second': 502.561, 'test_steps_per_second': 0.628}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,46.251,24.7122,16.9139,9.7393,46.251,34.7251,32.9816,33.0429,26.8635,25.0309,24.2147,24.876,26.8635,27.5939,29.1346,31.3642,32.181,42.1646,45.2166,0.0698,1559.4997,502.561,0.628


In [4]:
#wikiseealso
m = {'test_loss': 0.1637713611125946, 'test_P@1': 0.48389713545334195, 'test_P@10': 0.14728445483470948, 'test_P@3': 0.31811771775172687, 'test_P@5': 0.23777934259078015, 'test_N@1': 0.4838971495628357, 'test_N@10': 0.5204978585243225, 'test_N@3': 0.48093181848526, 'test_N@5': 0.4956223964691162, 'test_PSP@1': 0.3471672765048718, 'test_PSP@10': 0.47972551736149643, 'test_PSP@3': 0.38718797803337185, 'test_PSP@5': 0.4206066509767296, 'test_PSN@1': 0.3471672534942627, 'test_PSN@10': 0.4514100253582001, 'test_PSN@3': 0.39258873462677, 'test_PSN@5': 0.4197971522808075, 'test_R@200': 0.7652431698696025, 'test_R@10': 0.5857672396317298, 'test_R@100': 0.7389251695375327, 'test_runtime': 354.4095, 'test_samples_per_second': 500.875, 'test_steps_per_second': 0.313}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,48.3897,31.8118,23.7779,14.7284,48.3897,48.0932,49.5622,52.0498,34.7167,38.7188,42.0607,47.9726,34.7167,39.2589,41.9797,45.141,58.5767,73.8925,76.5243,0.1638,354.4095,500.875,0.313


In [9]:
#wikiseealso

# test_enc + 0.9 * test_teacher_lbl
m = {'P@1': 0.4926175252795538, 'P@10': 0.15095175055618268, 'P@3': 0.32434629937394543, 'P@5': 0.24306340309268976, 'N@1': 0.49261752, 'N@10': 0.5301417, 'N@3': 0.48901075, 'N@5': 0.5043292, 'PSP@1': 0.34847975871404174, 'PSP@10': 0.4925119161157266, 'PSP@3': 0.39387665692628143, 'PSP@5': 0.4300987601327263, 'PSN@1': 0.34847975, 'PSN@10': 0.45926732, 'PSN@3': 0.39760143, 'PSN@5': 0.42626452, 'R@200': 0.7865898066958945, 'R@10': 0.5977345456137035, 'R@100': 0.7590797989587106}
display_metric(m, remove_prefix=False)

# test_enc + 0.6 * test_centroid_repr
m = {'P@1': 0.4930625580936822, 'P@10': 0.14932766245095766, 'P@3': 0.32266005689677707, 'P@5': 0.24090020561642803, 'N@1': 0.49306256, 'N@10': 0.52717984, 'N@3': 0.48790085, 'N@5': 0.50213796, 'PSP@1': 0.36139826033163214, 'PSP@10': 0.48929007965248594, 'PSP@3': 0.39825073226521923, 'PSP@5': 0.4301212071581012, 'PSN@1': 0.36139825, 'PSN@10': 0.4610039, 'PSN@3': 0.4035537, 'PSN@5': 0.42961562, 'R@200': 0.7682362092089037, 'R@10': 0.5909517316098264, 'R@100': 0.7460220673711099}
display_metric(m, remove_prefix=False)

# test_clf + 0.9 * test_centroid_fused_repr
m = {'P@1': 0.4962341210601921, 'P@10': 0.15043742782289565, 'P@3': 0.3246880545307812, 'P@5': 0.24256879700306508, 'N@1': 0.49623412, 'N@10': 0.53012913, 'N@3': 0.4905924, 'N@5': 0.5049278, 'PSP@1': 0.3615449989425466, 'PSP@10': 0.49166717282959804, 'PSP@3': 0.39935205078505587, 'PSP@5': 0.4317239936704363, 'PSN@1': 0.36154497, 'PSN@10': 0.46223307, 'PSN@3': 0.40426677, 'PSN@5': 0.4305397, 'R@200': 0.7869761880866443, 'R@10': 0.5941071112220179, 'R@100': 0.7501463400002143}
display_metric(m, remove_prefix=False)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.2618,32.4346,24.3063,15.0952,49.2618,48.9011,50.4329,53.0142,34.848,39.3877,43.0099,49.2512,34.848,39.7601,42.6265,45.9267,59.7735,75.908,78.659


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.3063,32.266,24.09,14.9328,49.3063,48.7901,50.2138,52.718,36.1398,39.8251,43.0121,48.929,36.1398,40.3554,42.9616,46.1004,59.0952,74.6022,76.8236


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,49.6234,32.4688,24.2569,15.0437,49.6234,49.0592,50.4928,53.0129,36.1545,39.9352,43.1724,49.1667,36.1545,40.4267,43.054,46.2233,59.4107,75.0146,78.6976


In [5]:
#wikipedia

### `momos M2`

In [9]:
#wikiseealsotitles
m = {'test_loss': 0.08250453323125839, 'test_P@1': 0.3504098245218714, 'test_P@10': 0.11164014308651794, 'test_P@3': 0.2374109230206094, 'test_P@5': 0.178951637889937, 'test_N@1': 0.3504098355770111, 'test_N@10': 0.3743511736392975, 'test_N@3': 0.3474583923816681, 'test_N@5': 0.3568243980407715, 'test_PSP@1': 0.2752314455371148, 'test_PSP@10': 0.371782579869313, 'test_PSP@3': 0.3047629916581384, 'test_PSP@5': 0.328367531139476, 'test_PSN@1': 0.2752314507961273, 'test_PSN@10': 0.34207022190093994, 'test_PSN@3': 0.3036549389362335, 'test_PSN@5': 0.320774644613266, 'test_R@200': 0.5670612413135152, 'test_R@10': 0.41908470502431966, 'test_R@100': 0.5416234576180196, 'test_runtime': 55.8045, 'test_samples_per_second': 3181.014, 'test_steps_per_second': 1.989}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,35.041,23.7411,17.8952,11.164,35.041,34.7458,35.6824,37.4351,27.5231,30.4763,32.8368,37.1783,27.5231,30.3655,32.0775,34.207,41.9085,54.1623,56.7061,0.0825,55.8045,3181.014,1.989


In [18]:
#wikiseealsotitles

# test_clf + 0.2 * test_enc
m = {'P@1': 0.35120412359518915, 'P@10': 0.11225980903024012, 'P@3': 0.23841365518408894, 'P@5': 0.17970988367198423, 'N@1': 0.35120413, 'N@10': 0.37621465, 'N@3': 0.34886736, 'N@5': 0.35838717, 'PSP@1': 0.2775433379150663, 'PSP@10': 0.3749818434087103, 'PSP@3': 0.3073157950437174, 'PSP@5': 0.33099236118375347, 'PSN@1': 0.27754334, 'PSN@10': 0.344846, 'PSN@3': 0.30608308, 'PSN@5': 0.32330644, 'R@200': 0.568667339855646, 'R@10': 0.4216408496040447, 'R@100': 0.5477761420753614} 
display_metric(m, remove_prefix=False)

# test_clf + 0.2 * test_centroid_repr
m = {'P@1': 0.3512829901698448, 'P@10': 0.11263329859450111, 'P@3': 0.23842116628643664, 'P@5': 0.18003098329736691, 'N@1': 0.35128298, 'N@10': 0.3764666, 'N@3': 0.34867075, 'N@5': 0.35832953, 'PSP@1': 0.27860606061826354, 'PSP@10': 0.3763109585337449, 'PSP@3': 0.3079967554608065, 'PSP@5': 0.3319881798798843, 'PSN@1': 0.27860606, 'PSN@10': 0.3453594, 'PSN@3': 0.30648392, 'PSN@5': 0.3236756, 'R@200': 0.5683758520722021, 'R@10': 0.4221968231113175, 'R@100': 0.5490152025392615}
display_metric(m, remove_prefix=False)

# test_clf + 0.2 * test_centroid_fused_repr
m = {'P@1': 0.3517336563107343, 'P@10': 0.11297862152495697, 'P@3': 0.23898449896254897, 'P@5': 0.18057628932784378, 'N@1': 0.35173365, 'N@10': 0.37736115, 'N@3': 0.3494572, 'N@5': 0.35922197, 'PSP@1': 0.2781064653233376, 'PSP@10': 0.37673358347444735, 'PSP@3': 0.30804218106809594, 'PSP@5': 0.3322534324269027, 'PSN@1': 0.27810645, 'PSN@10': 0.3454442, 'PSN@3': 0.30646953, 'PSN@5': 0.32375866, 'R@200': 0.5687609045426391, 'R@10': 0.423278324106155, 'R@100': 0.5505767445182107}
display_metric(m, remove_prefix=False)

# test_clf + 0.2 * test_teacher_lbl
m = {'P@1': 0.35111962369377236, 'P@10': 0.11297749485960493, 'P@3': 0.23933564299732582, 'P@5': 0.18057966932390176, 'N@1': 0.35111964, 'N@10': 0.37655032, 'N@3': 0.3491049, 'N@5': 0.35857683, 'PSP@1': 0.27534147796991787, 'PSP@10': 0.3768198420841615, 'PSP@3': 0.3075191100491229, 'PSP@5': 0.3317501507879783, 'PSN@1': 0.27534145, 'PSN@10': 0.34427264, 'PSN@3': 0.30513218, 'PSN@5': 0.3224494, 'R@200': 0.5691823668902801, 'R@10': 0.42222068634594395, 'R@100': 0.5499031959716303}
display_metric(m, remove_prefix=False)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,35.1204,23.8414,17.971,11.226,35.1204,34.8867,35.8387,37.6215,27.7543,30.7316,33.0992,37.4982,27.7543,30.6083,32.3306,34.4846,42.1641,54.7776,56.8667


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,35.1283,23.8421,18.0031,11.2633,35.1283,34.8671,35.833,37.6467,27.8606,30.7997,33.1988,37.6311,27.8606,30.6484,32.3676,34.5359,42.2197,54.9015,56.8376


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,35.1734,23.8984,18.0576,11.2979,35.1734,34.9457,35.9222,37.7361,27.8106,30.8042,33.2253,37.6734,27.8106,30.647,32.3759,34.5444,42.3278,55.0577,56.8761


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200
0,35.112,23.9336,18.058,11.2977,35.112,34.9105,35.8577,37.655,27.5341,30.7519,33.175,37.682,27.5341,30.5132,32.2449,34.4273,42.2221,54.9903,56.9182


In [10]:
#wikititles

In [ ]:
#wikiseealso

In [11]:
#wikipedia

### `momos oracle M1`

Metrics with ground truth metadata.

In [57]:
# wikiseealsotitles
m = {'test_loss': 0.19825103878974915, 'test_P@1': 0.37074050080274906, 'test_P@10': 0.1188265780356551, 'test_P@3': 0.25063421870457464, 'test_P@5': 0.18944990564195896, 'test_N@1': 0.37074050307273865, 'test_N@10': 0.4057261645793915, 'test_N@3': 0.3724217712879181, 'test_N@5': 0.3848222494125366, 'test_PSP@1': 0.2962928678575879, 'test_PSP@10': 0.3982671959567921, 'test_PSP@3': 0.32501365704165813, 'test_PSP@5': 0.35058293049541067, 'test_PSN@1': 0.29629287123680115, 'test_PSN@10': 0.37048059701919556, 'test_PSN@3': 0.3266501724720001, 'test_PSN@5': 0.3462205231189728, 'test_R@200': 0.6369637154078921, 'test_R@10': 0.4608445217062401, 'test_R@100': 0.6069939450201214, 'test_runtime': 245.6463, 'test_samples_per_second': 722.645, 'test_steps_per_second': 0.452}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,37.0741,25.0634,18.945,11.8827,37.0741,37.2422,38.4822,40.5726,29.6293,32.5014,35.0583,39.8267,29.6293,32.665,34.6221,37.0481,46.0845,60.6994,63.6964,0.1983,245.6463,722.645,0.452


In [62]:
# wikiseealsotitles
# number of categories: 5
m = {'test_loss': 0.19786709547042847, 'test_P@1': 0.3702391347210095, 'test_P@10': 0.11890882460636706, 'test_P@3': 0.25066050756279223, 'test_P@5': 0.18955468551971572, 'test_N@1': 0.37023913860321045, 'test_N@10': 0.405764102935791, 'test_N@3': 0.3722630441188812, 'test_N@5': 0.38483309745788574, 'test_PSP@1': 0.2960730216869164, 'test_PSP@10': 0.398552853430692, 'test_PSP@3': 0.3251863600330204, 'test_PSP@5': 0.3508256123083663, 'test_PSN@1': 0.2960730195045471, 'test_PSN@10': 0.37052610516548157, 'test_PSN@3': 0.32662031054496765, 'test_PSN@5': 0.3462768495082855, 'test_R@200': 0.6386829313483762, 'test_R@10': 0.46123652095134443, 'test_R@100': 0.6081462389523458, 'test_runtime': 295.7903, 'test_samples_per_second': 600.138, 'test_steps_per_second': 0.375}
o = display_metric(m)

# number of categories: 10
m = {'test_loss': 0.19777467846870422, 'test_P@1': 0.3701490014928316, 'test_P@10': 0.11889079796073136, 'test_P@3': 0.2505403299252217, 'test_P@5': 0.1895377855394318, 'test_N@1': 0.37014901638031006, 'test_N@10': 0.4056752026081085, 'test_N@3': 0.37211939692497253, 'test_N@5': 0.38474544882774353, 'test_PSP@1': 0.2961520500099008, 'test_PSP@10': 0.39854476945783035, 'test_PSP@3': 0.3251778360242046, 'test_PSP@5': 0.35086973315236486, 'test_PSN@1': 0.2961520552635193, 'test_PSN@10': 0.3705241084098816, 'test_PSN@3': 0.3266165852546692, 'test_PSN@5': 0.346291184425354, 'test_R@200': 0.639163230813345, 'test_R@10': 0.46118741409794906, 'test_R@100': 0.6085032436962176, 'test_runtime': 390.59, 'test_samples_per_second': 454.479, 'test_steps_per_second': 0.284}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,37.0239,25.0661,18.9555,11.8909,37.0239,37.2263,38.4833,40.5764,29.6073,32.5186,35.0826,39.8553,29.6073,32.662,34.6277,37.0526,46.1237,60.8146,63.8683,0.1979,295.7903,600.138,0.375


,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,37.0149,25.054,18.9538,11.8891,37.0149,37.2119,38.4745,40.5675,29.6152,32.5178,35.087,39.8545,29.6152,32.6617,34.6291,37.0524,46.1187,60.8503,63.9163,0.1978,390.59,454.479,0.284


In [68]:
# wikititles
m = {'test_loss': 0.06927984207868576, 'test_P@1': 0.45808128429855194, 'test_P@10': 0.09496620703453507, 'test_P@3': 0.24056737986854726, 'test_P@5': 0.16484255680662488, 'test_N@1': 0.45808127522468567, 'test_N@10': 0.32611966133117676, 'test_N@3': 0.34089207649230957, 'test_N@5': 0.32468974590301514, 'test_PSP@1': 0.2692115310141104, 'test_PSP@10': 0.24277318787610716, 'test_PSP@3': 0.24363384608297156, 'test_PSP@5': 0.23566574430676646, 'test_PSN@1': 0.2692115306854248, 'test_PSN@10': 0.31049057841300964, 'test_PSN@3': 0.27161550521850586, 'test_PSN@5': 0.28750190138816833, 'test_R@200': 0.4766446025879506, 'test_R@10': 0.3208551847210097, 'test_R@100': 0.4399206996658419, 'test_runtime': 1554.1354, 'test_samples_per_second': 504.295, 'test_steps_per_second': 0.631}
display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,45.8081,24.0567,16.4843,9.4966,45.8081,34.0892,32.469,32.612,26.9212,24.3634,23.5666,24.2773,26.9212,27.1616,28.7502,31.0491,32.0855,43.9921,47.6645,0.0693,1554.1354,504.295,0.631


In [77]:
# wikititles
# number of categories: 5
m = {'test_loss': 0.07321329414844513, 'test_P@1': 0.4580889398693194, 'test_P@10': 0.0949674829629965, 'test_P@3': 0.24056057491675414, 'test_P@5': 0.16484026013539468, 'test_N@1': 0.4580889344215393, 'test_N@10': 0.3261234760284424, 'test_N@3': 0.3408890962600708, 'test_N@5': 0.3246893286705017, 'test_PSP@1': 0.2692090893675554, 'test_PSP@10': 0.2427803836190167, 'test_PSP@3': 0.2436241110461253, 'test_PSP@5': 0.2356614507681842, 'test_PSN@1': 0.269209086894989, 'test_PSN@10': 0.3104959726333618, 'test_PSN@3': 0.2716104984283447, 'test_PSN@5': 0.28750085830688477, 'test_R@200': 0.476652661222211, 'test_R@10': 0.3208593249395337, 'test_R@100': 0.439929277887563, 'test_runtime': 1074.2185, 'test_samples_per_second': 729.594, 'test_steps_per_second': 0.456}
o = display_metric(m)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,45.8089,24.0561,16.484,9.4967,45.8089,34.0889,32.4689,32.6123,26.9209,24.3624,23.5661,24.278,26.9209,27.161,28.7501,31.0496,32.0859,43.9929,47.6653,0.0732,1074.2185,729.594,0.456


### Teacher

In [12]:
models = ['NGAME (teacher)', 'RAMEN (baseline)']
datasets = ['LF-WikiSeeAlsoTitles-320K', 'LF-WikiTitles-500K', 'LF-WikiSeeAlso-320K', 'LF-Wikipedia-500K']

index = pd.MultiIndex.from_tuples([(d, m) for d in datasets for m in models], names=["Dataset", "Model"])

metric = [metrics(47.4794, 22.6355, 48.1811, 36.5976, 41.2696), metrics(35.2, 18.4, 36.5, 29.0,  34.5),
          metrics(66.0404, 32.1966, 53.4045, 38.8664, 42.5038), metrics(46.1, 17.4, 33.8, 30.5, 25.7),
          metrics(50.573, 24.322, 51.423, 36.779, 43.385), metrics(47.29, 23.37, 49.15, 33.81, 41.95),
          metrics(82.54, 48.759, 74.495, 46.612, 59.492), metrics(81.07, 50.1, 75.27, 50.94, 61.84)
         ]

display_results(metric, index)

P@1    P@5    N@5  PSP@1  PSP@5
Dataset                   Model                                              
LF-WikiSeeAlsoTitles-320K NGAME (teacher)   47.48  22.64  48.18  36.60  41.27
                          RAMEN (baseline)  35.20  18.40  36.50  29.00  34.50
LF-WikiTitles-500K        NGAME (teacher)   66.04  32.20  53.40  38.87  42.50
                          RAMEN (baseline)  46.10  17.40  33.80  30.50  25.70
LF-WikiSeeAlso-320K       NGAME (teacher)   50.57  24.32  51.42  36.78  43.38
                          RAMEN (baseline)  47.29  23.37  49.15  33.81  41.95
LF-Wikipedia-500K         NGAME (teacher)   82.54  48.76  74.50  46.61  59.49
                          RAMEN (baseline)  81.07  50.10  75.27  50.94  61.84

### LF-WikiSeeAlsoTitles-320K

In [10]:
dataset_name = 'LF-WikiSeeAlsoTitles-320K'
models = ['teacher', 'momos-oracle', 'oak-oracle', 'momos', 'oak']

index = pd.MultiIndex.from_tuples([(dataset_name, o) for o in models], names=["Dataset", "Model"])

metric = [metrics(47.4794, 22.6355, 48.1811, 36.5976, 41.2696),
          metrics(40.92, 19.97, 40.889, 30.1417, 35.3729),
          metrics(38.92, 19.35, 40.35, 29.69, 34.85),
          metrics(34.87, 17.95, 35.86, 28.39, 33.6),
          metrics(32.75, 16.64, 33.65, 25.67, 30.51)]

display_results(metric, index)

P@1    P@5    N@5  PSP@1  PSP@5
Dataset                   Model                                          
LF-WikiSeeAlsoTitles-320K teacher       47.48  22.64  48.18  36.60  41.27
                          momos-oracle  40.92  19.97  40.89  30.14  35.37
                          oak-oracle    38.92  19.35  40.35  29.69  34.85
                          momos         34.87  17.95  35.86  28.39  33.60
                          oak           32.75  16.64  33.65  25.67  30.51

### Results

In [21]:
dataset_name = ['LF-WikiSeeAlsoTitles-320K', 'LF-WikiTitles-500K', 'LF-WikiSeeAlso-320K']
models = ['teacher', 'momos', 'oak']

ll = [(d, o) for d in dataset_name for o in models]
ll[-1] = (ll[-1][0], '*'+ll[-1][1])

index = pd.MultiIndex.from_tuples(ll, names=["Dataset", "Model"])

metric = [
    metrics(47.48, 22.64, 48.18, 36.60, 41.27),
    metrics(34.87, 17.95, 35.86, 28.39, 33.60),
    metrics(32.75, 16.64, 33.65, 25.67, 30.51),
    
    metrics(66.0404, 32.1966, 53.4045, 38.8664, 42.5038),
    metrics(46.25, 16.91, 32.98, '-', '-'),
    metrics(43.80, 17.23, 32.94, 26.00, 24.42),

    metrics(50.37, 24.34, 51.38, 36.73, 43.51),
    metrics(49.62, 24.25, 50.49, 36.15, 43.17),
    metrics(48.57, 23.28, 49.16,33.92, 40.44),
]

display_results(metric, index)

P@1    P@5    N@5  PSP@1  PSP@5
Dataset                   Model                                     
LF-WikiSeeAlsoTitles-320K teacher  47.48  22.64  48.18   36.6  41.27
                          momos    34.87  17.95  35.86  28.39   33.6
                          oak      32.75  16.64  33.65  25.67  30.51
LF-WikiTitles-500K        teacher  66.04  32.20  53.40  38.87   42.5
                          momos    46.25  16.91  32.98      -      -
                          oak      43.80  17.23  32.94   26.0  24.42
LF-WikiSeeAlso-320K       teacher  50.37  24.34  51.38  36.73  43.51
                          momos    49.62  24.25  50.49  36.15  43.17
                          *oak     48.57  23.28  49.16  33.92  40.44

### Ablation

In [20]:
# Contribution of different loss terms
index = pd.MultiIndex.from_tuples([('LF-WikiSeeAlsoTitles-320K','OAK+data-term'), ('LF-WikiSeeAlsoTitles-320K', 'OAK+both'), ('LF-WikiSeeAlsoTitles-320K', 'OAK+label-term'),
                                  ('LF-WikiTitles-500K','OAK+data-term'), ('LF-WikiTitles-500K', 'OAK+both')], 
                                  names=["Dataset", "Loss function"])

metric = [metrics(34.87, 17.95, 35.86, 28.39, 33.60),
          metrics(32.51, 16.87, 34.14, 26.83, 31.51),
          metrics(31.10, 16.43, 33.42, 26.13, 30.50),
          metrics(45.0, 16.25, 31.95, 26.43, 23.39),
          metrics(43.53, 16.27, 31.53, 25.87, 23.34),]

display_results(metric, index)

P@1    P@5    N@5  PSP@1  PSP@5
Dataset                   Loss function                                    
LF-WikiSeeAlsoTitles-320K OAK+data-term   34.87  17.95  35.86  28.39  33.60
                          OAK+both        32.51  16.87  34.14  26.83  31.51
                          OAK+label-term  31.10  16.43  33.42  26.13  30.50
LF-WikiTitles-500K        OAK+data-term   45.00  16.25  31.95  26.43  23.39
                          OAK+both        43.53  16.27  31.53  25.87  23.34

In [22]:
# distillation with NGAME
index = pd.MultiIndex.from_tuples([('LF-WikiSeeAlsoTitles-320K', 'momos'),('LF-WikiSeeAlsoTitles-320K','NGAME')], names=["Dataset", "Model"])

metric = [metrics(34.87, 17.95, 35.86, 28.39, 33.60),
          metrics(32.58, 16.63, 33.58, 27.12, 31.62)]

display_results(metric, index)

P@1    P@5    N@5  PSP@1  PSP@5
Dataset                   Model                                   
LF-WikiSeeAlsoTitles-320K momos  34.87  17.95  35.86  28.39  33.60
                          NGAME  32.58  16.63  33.58  27.12  31.62

### Qualitative Analysis

In [1]:
from IPython.display import display,HTML

In [7]:
o = """<text style=color:red>data_input_text</text>: <text style=color:green>Grumman XP-50</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Lockheed XP-49', 'Westland Whirlwind', 'Gloster F.9/37', 'Grumman XP-50', 'Lockheed P-38 Lightning']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'Aircraft first flown in 1941', 'United States fighter aircraft 1940â\x80\x931949', 'Grumman aircraft']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Carrier-based aircraft', 'Cancelled military aircraft projects of the United States', 'Grumman aircraft']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Grumman XP-50</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Curtiss XP-46', 'Lockheed XP-49', 'Grumman XP-50', 'Curtiss XP-42', 'Grumman F2F']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'Aircraft first flown in 1941', 'United States fighter aircraft 1940â\x80\x931949', 'Grumman aircraft']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Carrier-based aircraft', 'Cancelled military aircraft projects of the United States', 'Grumman aircraft']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Grumman XP-50</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Focke-Wulf Fw 187', 'Westland Whirlwind', 'Kawasaki Ki-45', 'Gloster F.9/37', 'IMAM Ro.57', 'Lockheed P-38 Lightning', 'Mikoyan-Gurevich DIS', 'Tairov Ta-3', 'Grumman F7F Tigercat', 'Nakajima J1N', 'PZL.38 Wilk']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'Aircraft first flown in 1941', 'United States fighter aircraft 1940â\x80\x931949', 'Grumman aircraft']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Carrier-based aircraft', 'Cancelled military aircraft projects of the United States', 'Grumman aircraft']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Tangbe</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Tangbe', 'Mustang District', 'Kali Gandaki River', 'Upper Mustang', 'Gandaki River']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Populated places in Mustang District']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Populated places in Cameroon', 'Communes of Cameroon', 'Township-level divisions of Hebei']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Tangbe</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Tangbe', 'Desalpur', 'Vladivostok', 'Kitenge', 'List of currently erupting volcanoes']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Populated places in Mustang District']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Populated places in Cameroon', 'Communes of Cameroon', 'Township-level divisions of Hebei']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Tangbe</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Mustang District', 'Kali Gandaki Gorge', 'Kali Gandaki River', 'Upper Mustang', 'Gandaki River']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Populated places in Mustang District']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Populated places in Cameroon', 'Communes of Cameroon', 'Township-level divisions of Hebei']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>PZL-230 Skorpion</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Atlas Carver', 'Novi Avion', 'PZL-230 Skorpion', 'Embraer MFT-LF', 'Scaled Composites ARES']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Low-wing aircraft', 'Canard aircraft', 'Twinjets', 'PZL aircraft', 'Cancelled military aircraft projects of Poland']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'PZL aircraft']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>PZL-230 Skorpion</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['CASA C-101 Aviojet', 'Fouga CM.170 Magister', 'PZL M28 Skytruck', 'PZL-106 Kruk', 'PZL-230 Skorpion']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Low-wing aircraft', 'Canard aircraft', 'Twinjets', 'PZL aircraft', 'Cancelled military aircraft projects of Poland']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'PZL aircraft']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>PZL-230 Skorpion</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Atlas Carver', 'IAR 95', 'FMA SAIA 90', 'Novi Avion', 'Embraer MFT-LF', 'Scaled Composites ARES']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Low-wing aircraft', 'Canard aircraft', 'Twinjets', 'PZL aircraft', 'Cancelled military aircraft projects of Poland']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Low-wing aircraft', 'Twin-engined tractor aircraft', 'PZL aircraft']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Hearts of the World</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['The Kaiser, Beast of Berlin', 'To Hell with the Kaiser!', 'Hearts of the World', 'The Heart of Humanity', 'Over the Rhine']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['American films', 'American silent feature films', 'American black-and-white films', 'American drama films', 'Articles containing video clips', 'Paramount Pictures films', 'Films shot in London', 'Films shot in Los Angeles', 'Films shot in France', 'Films directed by D. W. Griffith', '1918 films', 'Famous Players-Lasky films', 'Films shot in England', 'American World War I propaganda films', '1910s war films']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Windows games', 'Paradox Interactive games', 'Video games developed in Sweden']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Hearts of the World</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Hearts of the World', 'List of grand strategy video games', 'List of Paradox Interactive games', 'Chronology of grand strategy video games', 'Hearts of Steel']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['American films', 'American silent feature films', 'American black-and-white films', 'American drama films', 'Articles containing video clips', 'Paramount Pictures films', 'Films shot in London', 'Films shot in Los Angeles', 'Films shot in France', 'Films directed by D. W. Griffith', '1918 films', 'Famous Players-Lasky films', 'Films shot in England', 'American World War I propaganda films', '1910s war films']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Windows games', 'Paradox Interactive games', 'Video games developed in Sweden']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Hearts of the World</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['The Kaiser, Beast of Berlin', 'To Hell with the Kaiser!', 'Yankee Doodle in Berlin', 'The Heart of Humanity', 'Over the Rhine', 'The False Faces', 'The Unpardonable Sin', 'The Battle Cry of Peace', 'Civilization']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['American films', 'American silent feature films', 'American black-and-white films', 'American drama films', 'Articles containing video clips', 'Paramount Pictures films', 'Films shot in London', 'Films shot in Los Angeles', 'Films shot in France', 'Films directed by D. W. Griffith', '1918 films', 'Famous Players-Lasky films', 'Films shot in England', 'American World War I propaganda films', '1910s war films']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Windows games', 'Paradox Interactive games', 'Video games developed in Sweden']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Venezuelan protests (2014âpresent)</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Timeline of the 2014 Venezuelan protests', 'Timeline of the 2015 Venezuelan protests', 'Timeline of the 2017 Venezuelan protests', 'Timeline of the 2018 Venezuelan protests', 'Timeline of the 2019 Venezuelan protests']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Articles containing video clips', 'Protests in Venezuela', 'Riots and civil disorder in Venezuela', '21st century in Caracas', '2014 protests', '2015 protests', '2014 riots', 'Venezuelan protests (2014â\x80\x93present)', '2014 in Venezuela', '2016 protests', '2017 protests', '2017 in Venezuela', 'Ongoing protests', '2016 in Venezuela', '2015 in Venezuela']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Protests in Venezuela', 'Venezuelan protests (2014â\x80\x93present)', 'Venezuela-related lists']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Venezuelan protests (2014âpresent)</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Bolivarian Revolution', '2014 Venezuelan protests', '2019 Venezuelan blackouts', 'Crisis in Venezuela', '2017 Venezuelan National Assembly attack']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Articles containing video clips', 'Protests in Venezuela', 'Riots and civil disorder in Venezuela', '21st century in Caracas', '2014 protests', '2015 protests', '2014 riots', 'Venezuelan protests (2014â\x80\x93present)', '2014 in Venezuela', '2016 protests', '2017 protests', '2017 in Venezuela', 'Ongoing protests', '2016 in Venezuela', '2015 in Venezuela']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Protests in Venezuela', 'Venezuelan protests (2014â\x80\x93present)', 'Venezuela-related lists']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Venezuelan protests (2014âpresent)</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Timeline of the 2014 Venezuelan protests', 'Timeline of the 2015 Venezuelan protests', 'Timeline of the 2017 Venezuelan protests', 'Timeline of the 2018 Venezuelan protests', 'Timeline of the 2019 Venezuelan protests']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Articles containing video clips', 'Protests in Venezuela', 'Riots and civil disorder in Venezuela', '21st century in Caracas', '2014 protests', '2015 protests', '2014 riots', 'Venezuelan protests (2014â\x80\x93present)', '2014 in Venezuela', '2016 protests', '2017 protests', '2017 in Venezuela', 'Ongoing protests', '2016 in Venezuela', '2015 in Venezuela']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Protests in Venezuela', 'Venezuelan protests (2014â\x80\x93present)', 'Venezuela-related lists']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Mecca2Medina</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Islamic music', 'DAM (band)', 'Native Deen', 'Dirty Kuffar', 'Mecca2Medina']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Converts to Islam', 'Living people', 'English Muslims', 'Performers of Islamic music', 'Musical groups established in 1996', 'English hip hop groups', 'Iqra Promotions artists']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Egyptian architecture', 'Buildings and structures in Mecca', 'Mecca']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Mecca2Medina</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Mosque', 'Delicate Arch', 'Hajj', 'Mecca2Medina', 'List of ziyarat locations']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Converts to Islam', 'Living people', 'English Muslims', 'Performers of Islamic music', 'Musical groups established in 1996', 'English hip hop groups', 'Iqra Promotions artists']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Egyptian architecture', 'Buildings and structures in Mecca', 'Mecca']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Mecca2Medina</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Black British', 'List of converts to Islam', 'Islamic music', 'Nasheed', 'DAM (band)', 'British hip hop', 'Native Deen', 'Dirty Kuffar']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Converts to Islam', 'Living people', 'English Muslims', 'Performers of Islamic music', 'Musical groups established in 1996', 'English hip hop groups', 'Iqra Promotions artists']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Egyptian architecture', 'Buildings and structures in Mecca', 'Mecca']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Sayn</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Sayn-Wittgenstein-Berleburg', 'Sayn-Wittgenstein-Hohenstein', 'Sayn-Wittgenstein-Sayn', 'Sponheim-Sayn', 'Sayn-Homburg']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['States and territories established in the 11th century', 'History of the Rhineland', 'Former states and territories of Rhineland-Palatinate', 'History of the Westerwald']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Communes of PyrÃ©nÃ©es-Atlantiques', 'Populated places in Badakhshan Province', 'Irish-language feminine given names']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Sayn</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Balkh Province', 'Khar Road', 'Faysh Khabur', 'Narjis', 'Hashim']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['States and territories established in the 11th century', 'History of the Rhineland', 'Former states and territories of Rhineland-Palatinate', 'History of the Westerwald']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Communes of PyrÃ©nÃ©es-Atlantiques', 'Populated places in Badakhshan Province', 'Irish-language feminine given names']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Sayn</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['Sayn-Wittgenstein-Berleburg', 'Sayn-Wittgenstein-Hachenburg', 'Sayn-Wittgenstein-Hohenstein', 'Sayn-Wittgenstein-Karlsburg', 'Sayn-Wittgenstein-Ludwigsburg', 'Sayn-Wittgenstein-Sayn', 'Sayn-Wittgenstein-Sayn-Altenkirchen', 'Sayn-Wittgenstein-Vallendar', 'Sponheim-Sayn', 'Sayn-Homburg']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['States and territories established in the 11th century', 'History of the Rhineland', 'Former states and territories of Rhineland-Palatinate', 'History of the Westerwald']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Communes of PyrÃ©nÃ©es-Atlantiques', 'Populated places in Badakhshan Province', 'Irish-language feminine given names']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>List of highest domestic net collection of Hindi films</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['List of highest-grossing Indian films', '100 Crore Club', '1000 Crore Club', 'List of highest-grossing Indian films in overseas markets', 'List of highest-grossing films in India']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Lists of Indian films']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Bollywood', 'Cinema of India', 'Lists of Indian films']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>List of highest domestic net collection of Hindi films</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Bibliography of Hindi cinema', 'List of Bollywood films', 'List of highest-grossing Bollywood films', 'Hindi Cinema', 'Lists of Bollywood films']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Lists of Indian films']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Bollywood', 'Cinema of India', 'Lists of Indian films']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>List of highest domestic net collection of Hindi films</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['List of highest-grossing Indian films', '100 Crore Club', '1000 Crore Club', 'List of highest-grossing Indian films in overseas markets', 'List of highest-grossing films in India']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Lists of Indian films']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Bollywood', 'Cinema of India', 'Lists of Indian films']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Circle</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Apeirogon', 'Unit circle', 'Bankoff circle', 'Woo circles', 'Circle']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Curves', 'Circles', 'Pi', 'Conic sections', 'Elementary shapes']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Elementary geometry', 'Circles', 'Spherical trigonometry']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Circle</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['Apeirogon', 'Circle', 'Circumscribed circle', 'Triangle', 'Apeirogonal prism']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Curves', 'Circles', 'Pi', 'Conic sections', 'Elementary shapes']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Elementary geometry', 'Circles', 'Spherical trigonometry']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Circle</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>["Archimedes' twin circles", 'Affine sphere', 'Annulus (mathematics)', 'Sphere', 'Translation of axes', 'Unit circle', 'Apollonian circles', 'Chromatic circle', 'Ford circle', 'Circle of antisimilitude', 'Carlyle circle', 'Bankoff circle', 'Archimedean circle', 'Schoch circles', 'Woo circles', 'Nine-point circle', 'Excircle', 'Lester circle', 'Malfatti circles', 'Parry circle', 'Great circle', 'Villarceau circles']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Curves', 'Circles', 'Pi', 'Conic sections', 'Elementary shapes']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Elementary geometry', 'Circles', 'Spherical trigonometry']</text><br><br><text style=color:red>data_input_text</text>: <text style=color:green>Object 785</text><br><text style=color:red>lbl2data_input_text</text>: <text style=color:green>['Leopard 2-140', 'Pz 87-140', 'Object 195', 'Type 99KM', 'Object 292']</text><br><text style=color:red>cat2data_input_text</text>: <text style=color:green>['Abandoned military projects of the Soviet Union', 'Main battle tanks of the Cold War', 'Cold War tanks of the Soviet Union', 'Trial and research tanks of the Soviet Union', 'Main battle tanks of the Soviet Union']</text><br><text style=color:red>lnk2data_input_text</text>: <text style=color:green>['Kosmos satellites', 'Supernova remnants', 'World War II submarines of Germany']</text><br><text style=color:black>data_input_text</text>: <text style=color:blue>Object 785</text><br><text style=color:black>lbl2data_input_text</text>: <text style=color:blue>['List of minor planets and comets visited by spacecraft', 'Interstellar comet', 'B612 Foundation', 'List of comets visited by spacecraft', '46610 BÃ\x83Â©sixdouze']</text><br><text style=color:black>cat2data_input_text</text>: <text style=color:blue>['Abandoned military projects of the Soviet Union', 'Main battle tanks of the Cold War', 'Cold War tanks of the Soviet Union', 'Trial and research tanks of the Soviet Union', 'Main battle tanks of the Soviet Union']</text><br><text style=color:black>lnk2data_input_text</text>: <text style=color:blue>['Kosmos satellites', 'Supernova remnants', 'World War II submarines of Germany']</text><br><text style=color:orange>data_input_text</text>: <text style=color:brown>Object 785</text><br><text style=color:orange>lbl2data_input_text</text>: <text style=color:brown>['CATTB', 'Leopard 2-140', 'Pz 87-140', 'Object 195', 'Type 99KM', 'Object 292']</text><br><text style=color:orange>cat2data_input_text</text>: <text style=color:brown>['Abandoned military projects of the Soviet Union', 'Main battle tanks of the Cold War', 'Cold War tanks of the Soviet Union', 'Trial and research tanks of the Soviet Union', 'Main battle tanks of the Soviet Union']</text><br><text style=color:orange>lnk2data_input_text</text>: <text style=color:brown>['Kosmos satellites', 'Supernova remnants', 'World War II submarines of Germany']</text>"""

In [8]:
HTML(o)